In [1]:
import math

def compute_movement(current_pos, obj, current_angle_deg): 
    # current_angle_deg = angle between thymio front axis and x-axis
    # Extracting coordinates from current_pos and obj
    current_x, current_y = current_pos
    obj_x, obj_y = obj

    # Calculate distance between current position and the target object
    distance = math.sqrt((obj_x - current_x)**2 + (obj_y - current_y)**2)

    # Calculate angle in radians
    angle_radians = math.atan2(obj_y - current_y, obj_x - current_x)
    #angle_radians = math.atan2(obj_y , obj_x)
    #if (obj_y - current_y) <0 and obj_x - current_x < 0:£
    #    angle_radians = -angle_radians
    #print(obj_y - current_y)
    #print(obj_x - current_x)

    # Convert angle to degrees
    angle_degrees = math.degrees(angle_radians) - current_angle_deg


    
    print(f"Distance: {distance}", f"degrees: {angle_degrees}")
    return distance, angle_degrees
    

# Example usage
compute_movement([0, 0], [1, 1], -90)
    

    

Distance: 1.4142135623730951 degrees: 135.0


(1.4142135623730951, 135.0)

In [2]:
#USED TO CHECK FOR COLLISION AT REGULAR INTERVALS, Source : ex week 8
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

In [3]:
from tdmclient import ClientAsync, aw

import time
import numpy as np
import asyncio
Ts = 0.1
SPEED = 100
SEUIL_OBSTACLE = 100
SEUIL_KIDNAPPED = 100
#VALUES THAT NEED TO BE TUNED BEGIN: 
##We add correction factors to tune the speed symmetry
SPEED_LEFT = SPEED +1 
SPEED_RIGHT = SPEED

thymio_speed_to_mms = 19.73913043478261/50 ##TO FIND THE MM/s speed, divide motors_speed by this, thix value has to be tuned for each thymio
rotation_factor = 110*np.pi/360 
#VALUES TO BE TUNED END

#Global variables
obstacle_detected = 0
goal_reached = 0
kidnapped = 0
def get_horizontal():
    global obstacle_detected, kidnapped, rotation_done
    prox_values = node["prox.horizontal"][:5]
    #print(list(prox_values))
    if obstacle_detected ==0 and rotation_done == 1:
        obstacle_detected = any(value > SEUIL_OBSTACLE for value in prox_values)
        if obstacle_detected:
            rotation_done = 0
    ground_values = node["prox.ground.reflected"]
    #print(list(ground_values))
    
    if ground_values[0] < SEUIL_KIDNAPPED or ground_values[1] < SEUIL_KIDNAPPED:
        kidnapped = True 
        print(f"kidnapped")
    else:
        kidnapped = False
    

   
async def drive(): 
    v = {
        "motor.left.target": [int(SPEED_LEFT)],
        "motor.right.target": [int(SPEED_RIGHT)],
    }
    await node.set_variables(v)
async def stop(): 
    v = {
        "motor.left.target": [0],
        "motor.right.target": [0],
    }
    await node.set_variables(v)
#Adapted from moodle forum thread : 
async def drive_for_seconds(client, seconds):
#just drives straight for given seconds
# input: async client, seconds
    global obstacle_detected, kidnapped
    await node.wait_for_variables()
    rt = RepeatedTimer(Ts, get_horizontal)
    # Set the motor speeds
    v = {
        "motor.left.target": [int(SPEED_LEFT)],
        "motor.right.target": [int(SPEED_RIGHT)],
    }
    await node.set_variables(v)
    start_time = time.monotonic()
    while time.monotonic() - start_time < seconds:
        await node.wait_for_variables({"prox.horizontal"})
        
        #get_horizontal()
        await client.sleep(0.01)
        if obstacle_detected:
            # Si un obstacle est détecté, arrêtez les moteurs
            v_stop = {
                "motor.left.target": [int(0)],
                "motor.right.target": [int(0)],
            }
            await node.set_variables(v_stop)
    
            #obstacle_detected = False  # Réinitialisez la variable d'obstacle détecté
            rt.stop()
            break  # Sortez de la boucle si un obstacle est détecté
        await asyncio.sleep(Ts)

    # Stop the motors
    v_stop = {
        "motor.left.target": [int(0)],
        "motor.right.target": [int(0)],
    }
    rt.stop()
    await node.set_variables(v_stop)
async def rotate_for_seconds(client, seconds, degrees):
##Rotates during a number of seconds
##input : async client, number of seconds, degrees (used to know if he turns clockwise or counter-clockwise
    global obstacle_detected
    await node.wait_for_variables()
    rt2 = RepeatedTimer(Ts, get_horizontal)
    if degrees <=0:
        v = {
            "motor.left.target": [int(-SPEED_LEFT)],
            "motor.right.target": [int(SPEED_RIGHT)],
        }
        await node.set_variables(v)
        start_time = time.monotonic()
        while time.monotonic() - start_time < seconds:
            await node.wait_for_variables({"prox.horizontal"})
        
            await client.sleep(0.01)
            if obstacle_detected or kidnapped:
                # Si un obstacle est détecté, arrêtez les moteurs
                v_stop = {
                    "motor.left.target": [int(0)],
                    "motor.right.target": [int(0)],
                }
                await node.set_variables(v_stop)
                
                ##obstacle_detected = False  # Réinitialisez la variable d'obstacle détecté
                print(f"obstacle detected")
                break  # Sortez de la boucle si un obstacle est détecté
            await asyncio.sleep(Ts)
    
        # Stop the motors
        v_stop = {
            "motor.left.target": [int(0)],
            "motor.right.target": [int(0)],
        }
        rt2.stop()
        await node.set_variables(v_stop)
    else:
        v = {
            "motor.left.target": [int(SPEED_LEFT)],
            "motor.right.target": [int(-SPEED_RIGHT)],
        }
        await node.set_variables(v)
        start_time = time.monotonic()
        while time.monotonic() - start_time < seconds:
            await node.wait_for_variables({"prox.horizontal"})
        
            #get_horizontal()
            await client.sleep(0.001)
            if obstacle_detected or kidnapped:
                # Si un obstacle est détecté, arrêtez les moteurs
                v_stop = {
                    "motor.left.target": [int(0)],
                    "motor.right.target": [int(0)],
                }
                await node.set_variables(v_stop)
                
                ##obstacle_detected = False  # Réinitialisez la variable d'obstacle détecté
                break  # Sortez de la boucle si un obstacle est détecté
            #await asyncio.sleep(Ts)
    
        # Stop the motors
        v_stop = {
            "motor.left.target": [int(0)],
            "motor.right.target": [int(0)],
        }
        rt2.stop()
        await node.set_variables(v_stop)
async def rotate_angle(angle):
     global rotation_done
     if angle!= 0:
        disttorotate = abs(angle*rotation_factor)
        secondstorotate = disttorotate/(SPEED*thymio_speed_to_mms)
        rotate_task = asyncio.create_task(rotate_for_seconds(client, secondstorotate, np.sign(angle)))
        await rotate_task
        rotation_done = 1
         
    
async def print_elapsed_time(seconds, interval=0.5):
##Testing function to know elapsed-time, no practical use apart from calibration, to delete later
    start_time = time.monotonic()
    while True:
        elapsed = time.monotonic() - start_time
        if elapsed > seconds:
            break
        print(f"Elapsed time: {elapsed:.4f} seconds")
        await asyncio.sleep(interval)
async def road_trip(nodes_to_visit, current_pos, initial_angle):
##Makes the robot visit nodes one after another
## input: nodes_to_visit:list of nodes containing [x,y] coordinates, 
##        current_pos : [x,y] coordinates of current position, 
##        initial_angle : current angle between thymio sagittal axis and x-axis
    global obstacle_detected, goal_reached
    previous_angle = 0;
    for i in range(len(nodes_to_visit)):
        if obstacle_detected or kidnapped: 
            break
        else: 
            if i == 0:
             dist, angle = compute_movement(current_pos, nodes_to_visit[i], initial_angle)
             previous_angle = angle + initial_angle
            else: 
             dist, angle = compute_movement(nodes_to_visit[i-1], nodes_to_visit[i], previous_angle)
             previous_angle = angle + previous_angle
            await go_to(dist, angle)
            if i == len(nodes_to_visit)-1:
                goal_reached = True
                print(f"goal_reached")
        
    
    
async def go_to(dist, angle):
## Makes the thymio rotate of given angle, then go straight for distance dist, only meant to be used through road_trip function
## Input: dist : distance to travel in mm
##        angle : angle to rotate before going forward

    secondstodrive = dist/(SPEED*thymio_speed_to_mms)
    if angle!= 0:
        disttorotate = abs(angle*rotation_factor)
        secondstorotate = disttorotate/(SPEED*thymio_speed_to_mms)
        rotate_task = asyncio.create_task(rotate_for_seconds(client, secondstorotate, np.sign(angle)))
        await rotate_task
    
    drive_task = asyncio.create_task(drive_for_seconds(client, secondstodrive))
    
    #print_task = asyncio.create_task(print_elapsed_time(secondstodrive))
    
    # Wait for both tasks to complete
    await drive_task
    
    #await print_task

def close_coordinates(x, y, w, z, tolerance):
    distance = math.sqrt((w - x)**2 + (z - y)**2)
    return distance <= tolerance

## Example main program, all these lines are necessary
### This implementation will not be in the final project, to delete later

In [ ]:
from tdmclient import ClientAsync, aw
#from ComputerVision import Vision
import time
import cv2
#vis = Vision()
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()
rotation_done = 1
#Define the trip the robot has to make here : 
trip = ([0,100],[200,100], [200,300],[300,300], [300,0], [0,0])
trip2 = ([0,100], [0,200])
trip3 = ([100,0],[100,100],[0,0])
async def main():    
    global goal_reached, obstacle_detected
    while not goal_reached : 
        
 
        #vis.update()
        #trip = vis.find_shortest_path()
        await road_trip(trip3, [0,0], 360)
        if(obstacle_detected ): 
            await client.sleep(1)
            print(f"REPLANNING due to obstacle")
            obstacle_detected = False
            #vis.update()
            #trip = vis.find_shortest_path()
        #await road_trip(trip2, [vis.robot.x,vis.robot.y], vis.robot.angle)
        while kidnapped : 
            await client.sleep(1)
            get_horizontal()
            if kidnapped == 0:
                await client.sleep(1)
                print(f"REPLANNING due to kidnapping")
                #vis.update()
                #trip = vis.find_shortest_path() 
await main()
obstacle_detected = 0
goal_reached = 0
kidnapped = 0
await node.unlock()

Quick explaination on how we computed the seconds required to rotate of x degrees


You can use the following formula to calculate the number of wheel rotations \(T\) required for one complete rotation:
$$T = \frac{\pi \times L}{\pi \times D} $$
We measured L = 9.5cm (distance between wheels) and D = 4.2 cm (wheel diameter), but this value is meant to be tuned
we want to know the number of degrees the wheels will turn for the vehicle to complete one full rotation, we can use the formula:
$$\text{Number of degrees} = T \times 360 $$

Substitute the value of \(T\) from the previous formula:
$$\text{Number of degrees} = \frac{L}{D} \times 360 = 814.286$$

Then from this we can compute the required distance to travel for a given wanted rotation: 
$$\frac{814.286}{95*\pi} = \frac{wanted rotation°}{distance} \implies distance = \frac{\text{wanted rotation°} \times 95 \pi}{814.286}$$
From which we compute the seconds required:
$$\text{Seconds of rotation} =  \frac{distance}{SPEED \times conversionfactor}$$
In practice, the values L and D have to be tuned.


How I see it: 
Initially, we get the robot's pose and angle
We obtain the desired path
The robot orients itself, then moves towards the target.
Every few seconds, we re-measure the robot's position, apply the kalman filter and recalculate the angle it should make (correct its trajectory).
this means: just orient it and make it go to the desired node.
If the robot's x and y estimates are within a few cm of the node's, then we change our objective.
etc etc. 
And when the robot is kidnapped: we wait to see it again, and then start all over again.
If it detects an obstacle: we recalculate its trajectory, rotates till it does not see an obstacle anymore
# Final main loop putting all things together

In [ ]:
import tdmclient.notebook
await tdmclient.notebook.start()

In [4]:
from tdmclient import ClientAsync, aw
from vision.ComputerVision import Vision
import time
import cv2
vis = Vision()
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()
path = []
target_node = 0
tolerance = 20 #TO TUNE
rotation_done = 1
last_image_time = time.time()
previous_pos = []

async def update_motion():
        global target_node, path, rotation_done, goal_reached
        await client.sleep(0.001) 
        if not obstacle_detected : 
        # Replace this with your actual task
            #vis.update()
            pass
        #Kalman filter here:
        #Kalman_Estimate = Kalman(robot.x, robot.y, robot.angle)
        if close_coordinates(vis.robot.x, vis.robot.y, path[target_node][0], path[target_node][1], tolerance):
            if target_node != (len(path))-1:
                await stop()
                target_node = target_node + 1
            else: 
                goal_reached = True
                return
        #if obstacle_detected: 
                #path = vis.find_shortest_path()
                #target_node = 0
                #obstacle_detected = 0
        if target_node != len(path): 
            objective_pos = path[target_node]
        estimate_angle_degrees = math.degrees(vis.robot.angle)
        print("Angle :", estimate_angle_degrees)
        dist, angle = compute_movement([vis.robot.x, vis.robot.y], objective_pos, -estimate_angle_degrees)
        print(f"Robotx: ", vis.robot.x, f"Roboty: ", vis.robot.y, f"Robotangle: ", estimate_angle_degrees)
        print(f"Objective_pos ", objective_pos )
        await rotate_angle(angle)
        
        await drive()
            
            
         # Sleep for 1 second
#Define the trip the robot has to make here : 
async def main():    
    global goal_reached, obstacle_detected, target_node, path, last_image_time, previous_pos
    previous_pos = [vis.robot.x, vis.robot.y, vis.robot.angle]
    vis.update(True)
    vis.show()
    while True:
        if time.time() - last_image_time < 0.5:
            continue
        vis.show()
        vis.update(True)
        last_image_time = time.time()
        if len(vis.shortest_path)>1:
            break
    
    while True:
        ##Find shortest path initially
        #print(time.time())
        if time.time() - last_image_time < 0.5:
            continue

        vis.show()
        vis.update()
        

        # Update the
        # last image acquisition time
        last_image_time = time.time()

        if cv2.waitKey(1) & 0xFF == ord('q') and len(vis.shortest_path)  > 1: 
            await stop()
            break
        if not (vis.found_robot and vis.found_goal):
            await stop()
            continue
        
        path = vis.shortest_path
        if len(path) >= 1:
            if not kidnapped : 
                v = { "leds.top" :[0, 0, 0]}
                await node.set_variables(v)
                await update_motion()
            else:
                await stop() #TO DELETE LATER
                await client.sleep(0.01)
                v = { "leds.top" :[32, 0, 0]}
                await node.set_variables(v)
                get_horizontal()
                #if not kidnapped: 
                #    vis.update()
                #    path = vis.find_shortest_path()
                #    target_node = 0
            if goal_reached == True: 
                await stop()
                break

await main()
obstacle_detected = 0
goal_reached = 0
kidnapped = 0
await node.unlock()
del vis

100%|██████████| 2/2 [00:00<00:00, 333.58it/s]


yes :  [[166 361]
 [201 181]]


100%|██████████| 1/1 [00:00<00:00, 200.06it/s]


Angle : -38.15722658736906
Distance: 183.8620406718037 degrees: -117.18342549689592
Robotx:  166.0 Roboty:  361.5 Robotangle:  -38.15722658736906
Objective_pos  [201 181]
obstacle detected


100%|██████████| 2/2 [00:00<00:00, 333.91it/s]


...

100%|██████████| 1/1 [00:00<00:00, 142.87it/s]


Angle : 78.3106308245608
Distance: 26.92582403567252 degrees: 10.112040310912619
Robotx:  191.0 Roboty:  206.0 Robotangle:  78.3106308245608
Objective_pos  [201 181]


100%|██████████| 1/1 [00:00<00:00, 250.23it/s]


Angle : 77.366638064725
Distance: 21.242645786248 degrees: 13.931689241802985
Robotx:  191.5 Roboty:  200.0 Robotangle:  77.366638064725
Objective_pos  [201 181]


100%|██████████| 1/1 [00:00<00:00, 200.01it/s]
